In [13]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [14]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26.4M/26.4M [00:05<00:00, 5.28MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 98.2kB/s]
100%|██████████| 4.42M/4.42M [00:03<00:00, 1.44MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 373kB/s]


In [15]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)


for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [16]:
device="cpu"

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model=NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [17]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [18]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [19]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [20]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.304486  [   64/60000]
loss: 2.294286  [ 6464/60000]
loss: 2.272556  [12864/60000]
loss: 2.272358  [19264/60000]
loss: 2.249222  [25664/60000]
loss: 2.214157  [32064/60000]
loss: 2.229100  [38464/60000]
loss: 2.187249  [44864/60000]
loss: 2.198145  [51264/60000]
loss: 2.157663  [57664/60000]
Test Error: 
 Accuracy: 38.3%, Avg loss: 2.153369 

Epoch 2
-------------------------------
loss: 2.160765  [   64/60000]
loss: 2.152739  [ 6464/60000]
loss: 2.099295  [12864/60000]
loss: 2.119624  [19264/60000]
loss: 2.059974  [25664/60000]
loss: 1.998065  [32064/60000]
loss: 2.028615  [38464/60000]
loss: 1.947880  [44864/60000]
loss: 1.964797  [51264/60000]
loss: 1.887511  [57664/60000]
Test Error: 
 Accuracy: 55.9%, Avg loss: 1.884255 

Epoch 3
-------------------------------
loss: 1.910700  [   64/60000]
loss: 1.880601  [ 6464/60000]
loss: 1.773620  [12864/60000]
loss: 1.820657  [19264/60000]
loss: 1.698702  [25664/60000]
loss: 1.654572  [32064/600

In [22]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [24]:
model = NeuralNetwork().to(device) # recreate model structure
model.load_state_dict(torch.load("model.pth", weights_only=True)) # load state dict into it

<All keys matched successfully>